In [ ]:
#TensorFlow/Keras 

In [2]:
# Importing all Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# special imports for CV and NLP
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)
# 2.7.0

2.8.0


In [ ]:
#Typical Neural Network Architectures with Sequential API
# 3 main architectures of neural networks are deep feed-forward
        #Deep Nueral Network, convolutional and recurrent nets:

#1 DNN:  simple sequence of fully connected (dense) layers + dropout, batch normalization
#2 CNN:image classification, object detection, semantic segmentation, and other computer vision tasks
#    CNN architecture is a sequence of convolutional-pooling blocks followed by a little fully connected network
#3 RNN: sequential data processing — NLP tasks and time-series predictions
#    NLP architecture is an embedding layer and a sequence of bidirectional LSTM layers
#    RNN for time-series predictions is typically one-directional, although using bidirectional layers may improve the quality too

In [ ]:
# DNN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu',
                          input_shape=(num_inputs, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='relu'),  # regression
])

In [ ]:
# CNN
model = tf.keras.models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu',
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')  # binary classification
    # multi-class classification
    # layers.Dense(num_classes, activation='softmax')
])


In [ ]:
# RNN for NLP
model = tf.keras.Sequential([
    layers.Embedding(vocab_size,
                     embedding_dim,
                     input_length=max_length),
    layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(16)),
    layers.Dropout(0.3),
    layers.Dense(16, activation='relu'),
    layers.Dense(6, activation='softmax')  # multiclass classification
])

# RNN for time series
model = tf.keras.Sequential([
    layers.Conv1D(filters=128, kernel_size=7,
                  strides=1, padding="causal",
                  activation="relu",
                  input_shape=(WINDOW_SIZE, 1)),
    # univariate time series - predict a value based on
    # 'WINDOW_SIZE' previous steps for 1 feature

    layers.LSTM(32, return_sequences=True),
    layers.LSTM(16, return_sequences=True),
    layers.Dense(16, activation="relu"),
    layers.Dense(1)  # predict one value
])

# explore your model structure
model.summary()

In [ ]:
#Compile the model 3 PARAMETERS: optimizer, the loss function, and the metrics 
#optimizer : Adam or RMSProp
#Loss function 
#MSE, MAE, or Huber loss for regression,
#binary cross-entropy loss for binary classification, and sparse categorical cross-entropy loss for multiclass classification tasks if your label is an integer (i.e. 1, 2, 3 for 3-class classification).
#Categorical cross-entropy loss is usually used when your labels are represented as one-hot encoded vectors (i.e. [1, 0, 0], [0, 1, 0] and [0, 0, 1] for 3-class classification).

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Typical optimizers:
# - tf.keras.optimizers.Adam(learning_rate=0.001)
# - tf.keras.optimizers.RMSprop(learning_rate=0.001)

# Typical classification losses:
# - tf.keras.losses.BinaryCrossentropy()
# - tf.keras.losses.CategoricalCrossentropy()        # y - one-hot
# - tf.keras.losses.SparseCategoricalCrossentropy()  # y - integer

# Typical regression losses:
# - tf.keras.losses.MeanSquaredError()
# - tf.keras.losses.MeanAbsoluteError()
# - tf.keras.losses.Huber()

# Typical metrics
# - ['accuracy'] # for classification
# - ['mse', 'mae'] # for regression

In [ ]:
#Train the model
 #1 regular data in the form of arrays or tensors
 #2 ImageDataGenerator for computer vision
 #3  padded sequences for NLP
    
    #CALBACKS 
        #ModelCheckpoint (saves your model during its training)
        #EarlyStopping (stops the training process if the loss has stopped improving)
# callbacks
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath='data/model_callback.h5',
    save_weights_only=True,
    monitor='val_loss',  # 'val_accuracy'
    mode='min',  # 'max'
    save_best_only=True,
    verbose=1)

earlystop_cb = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=5,
    verbose=1)

callbacks = [earlystop_cb, checkpoint_cb]

# ------------------------------------------------------
# model.fit

# from from regular array data - see DNN architecture
history = model.fit(
    x=X_train,
    y=y_train,
    validation_split=0.1,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=callbacks,  # using callbacks
    verbose=1)

# from ImageDataGenerator - see CNN architecture
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    verbose=1)

# from padded sequences - see RNN for NLP architecture
history = model.fit(
    training_padded,
    training_labels,
    epochs=NUM_EPOCHS,
    validation_data=(validation_padded, validation_labels),
    verbose=1)

# from tf.data.Dataset - see RNN for time series architecture
history = model.fit(
    dataset,
    epochs=NUM_EPOCHS)

In [ ]:
# save  training results into a history variable to explore learning curves
# code is taken from:
# https://github.com/https-deeplearning-ai/tensorflow-1-public

def plot_learning_curves(history, parameter):
    """ 
    Plot learning curves from 'history' object.

    Parameters:
        history (History object): result of the `fit` method execution
        parameter (str): parameter to explore: 'accuracy', 'loss', etc.

    >>> history = model.fit(...)
    >>> plot_learning_curves(history, "loss")
    >>> plot_learning_curves(history, "accuracy")
    >>> plot_learning_curves(history, "mse")
    """

    plt.plot(history.history[parameter])
    plt.plot(history.history['val_'+parameter])
    plt.xlabel("Epochs")
    plt.ylabel(parameter)
    plt.legend([parameter, 'val_'+parameter])
    plt.show()


plot_learning_curves(history, "loss")
plot_learning_curves(history, "accuracy")
plot_learning_curves(history, "mse")

In [ ]:
#Evaluate the model --> model performance on previously unseen data
model.evaluate(X_test, y_test)

In [ ]:
#Save and load the model
model_path = 'data/model.h5'
model.save(model_path)
model = tf.keras.models.load_model(model_path)

# save using current date and time in filename
import time
current_time = time.strftime("%Y_%m_%d-%H_%M_%S")  # 2022_02_01-14_05_32
model_path = f'data/model_{current_time}.h5'
model.save(model_path)

# endddddddddddddd